# Collect data from the BigQuery database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

current_path = sys.path[0]
sys.path.append(current_path[:current_path.find('defi-measurement')] + "liquidity-distribution-history")

In [3]:
import os
from pool_state import v3Pool
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import json

import pandas as pd
from prisma import Prisma


from dotenv import load_dotenv


load_dotenv(override=True)

True

In [4]:
postgres_uri = os.getenv("POSTGRES_URI")

assert postgres_uri is not None, "Connection string to Postgres is not set"

In [5]:
pool_symbols = json.load(open("../addresses/pool_tokens.json", "r"))

# Take out the 10 first pools of the dictionary
# pool_symbols = {k: pool_symbols[k] for k in list(pool_symbols)[:10]}

In [6]:
client = Prisma()
await client.connect()

print(f"{await client.swap.count():,}")

364,367


## Get data for the 10 biggest pools by TVL

In [7]:
pool_addresses = [
    "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",  # USDC-ETH  0.05%
    "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed",   # WBTC-ETH  0.30%
    "0x5777d92f208679db4b9778590fa3cab3ac9e2168",  # DAI-USDC  0.01%
    "0x4585fe77225b41b697c938b018e2ac67ac5a20c0",  # WBTC-ETC  0.05%
    "0xc63b0708e2f7e69cb8a1df0e1389a98c35a76d52",  # FRAX-USDC 0.05%
    "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",  # USDC-ETH  0.30%
    "0x11b815efb8f581194ae79006d24e0d814b7697f6",  # ETH-USDT  0.05%
    "0x3416cf6c708da44db2624d63ea0aaef7113527c6",  # USDC-USDT 0.01%
    "0x7379e81228514a1d2a6cf7559203998e20598346",  # ETH/sETH2 0.30%
    "0x6c6bc977e13df9b0de53b251522280bb72383700",  # DAI-USDC  0.05%
]

In [9]:
it = tqdm(pool_addresses[:1])
for pool_address in it:
    it.set_description(pool_address)
    pool = v3Pool(pool_address, initialize=False, update=True, connStr=postgres_uri)

0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640:   0%|          | 0/1 [00:00<?, ?it/s]/Users/larsankile/Code/liquidity-distribution-history/pool_state.py:496: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  swaps = pd.read_sql(q, self.conn)
